<a href="https://colab.research.google.com/github/threegenie/similarity_search/blob/main/0404_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faiss와 SBERT를 이용한 시멘틱 검색
(참고 : https://wikidocs.net/162007)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


> package import

In [16]:
import pandas as pd
import numpy as np
import re
import time

> file import

사용한 데이터 : AIHUB 기사 텍스트 카테코리 분류 데이터 (https://aihub.or.kr/problem_contest/nipa-learning-platform/13)

In [4]:
df = pd.read_csv('/content/drive/My Drive/0404/text_classification.csv')

In [5]:
df.head()

,text,category
0,이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림...,연예/문화
1,"""지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요...",연예/문화
2,인천시 서구와 계양구에서 경기도 부천을 거쳐 서울 홍대까지 지하철이 건설될 것으로 ...,생활
3,김성태 자유한국당 의원이 최근 황운하 울산경찰청장과 벌인 ‘미친개’ 공방을 언급하며...,정치
4,"""지난해 사상 최대 실적을 거뒀던 우리 수출이 새해에도 좋은 흐름을 이어갔습니다. ...",스포츠


In [6]:
df.info

<bound method DataFrame.info of                                                     text category
0      이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림...    연예/문화
1      "지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요...    연예/문화
2      인천시 서구와 계양구에서 경기도 부천을 거쳐 서울 홍대까지 지하철이 건설될 것으로 ...       생활
3      김성태 자유한국당 의원이 최근 황운하 울산경찰청장과 벌인 ‘미친개’ 공방을 언급하며...       정치
4      "지난해 사상 최대 실적을 거뒀던 우리 수출이 새해에도 좋은 흐름을 이어갔습니다. ...      스포츠
...                                                  ...      ...
37643  뉴욕 대화 이후 북한이 추가 북미대화와 조기 협상재개를 강조하고 있는 반면, 미국은...       정치
37644  "프로농구 DB의 두경민이 생애 첫 최우수 선수의 영광을 안았습니다. 무명 설움을 ...      스포츠
37645  대한치과의사협회가 '공정위가 증거를 조작했다는 의혹이 있다'고 주장해 파문이 일고 ...       생활
37646  "세상의 종말을 느꼈다 필리핀 주민은 올해 가장 강력한 태풍인 &망쿳&의 위력을 이...    연예/문화
37647  [텐아시아=손예지 기자] /사진=김미화 SNS방송인 김미화가 부적절한 사과문에 대해...    연예/문화

[37648 rows x 2 columns]>

> 전처리

In [7]:
df['text'] = [re.sub('[^A-Za-z0-9가-힣 ]', '', s) for s in df['text']]
df.replace('', np.nan, inplace=True) 

> list 변환

In [8]:
data = df.text.to_list()

In [9]:
data[:5]

['이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 개국 공신이 되려는 몸부림으로 여길 수도 있다 하지만 지나친 부모의 교육열이 부른 무분별한 소비라는 지적도 만만치 않다 경기도 수원의 한 초등학교 교사는 수업 시간에 휴대전화로 문자를 주고받기에 혼냈더니 다음날 부모가 찾아와 내 아이 문제에 끼어들지 말라고 했다 요즘 초등학생들은 서구적 가치관도 전통적 가치관도 지니지 못한 경우가 많다고 고민을 토로했다 사진 5 BOX  요즘 아프트 놀이터에서 초등학생을 찾기는 쉽지 않다 지금 아이들은 학원과 PC방을 전전하고 있는지도 모른다 요즘 취학하는 아이들은 심각한 주의력 결핍증이 있다고 한다 유아기 때부터 온갖 시각 영상물에 노출된 탓이다 다양한 시각 정보를 처리하는 데 익숙하더라도 그 속에서 소비 욕구만 자극받는다면 헛일이다 인터넷 게임을 넓은 화면에서 즐기고 싶다며 벽걸이형 PDPPlasma Display Panel 텔레비전 구입을 요구하는 아이에게 무슨 말을 할 수 있을까 언제까지 자녀와의 친밀도를 경제력으로 확인할 수도 없는 일 아닌가 참교육을 위한 전국 학부모회 장은숙 사무처장은 올바른 디지털 소비문화 기준을 세워서 적용해야 한다 적어도 무엇 때문에 필요한지에 대해 식탁 토론이라도 벌이는 게 중요하다고 말한다 글 한겨레21바로가기',
 '지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요난민 체류 허가 뒤 맞이한 첫 주말 서울 도심에선 도로 하나를 사이에 두고난민에 대한 서로 다른 목소리가 터져나왔습니다강병수 기자가 현장에 다녀왔습니다 리포트 정부의 제주도 예멘 난민 일부에 대한 인도적 체류 허가 뒤 맞은 첫 주말 난민 찬성을 주장하는 집회에서는 인도적 체류만으로는 아직 갈 길이 멀다는 지적이 나왔습니다 이정원난민 찬성 집회 참가자  예멘 난민들에 대해 난민으로 인정할 수 없다고 결정했습니다 누구는 난민이고 누구는 난민이 아닐 수 없고 난민이 아니라는 정부의 판단이 잘못됐다고 생각합니다 문제는 난민 자체가 아니라 난민 혐오

> SBERT 임베딩(SentenceTransformer)

In [11]:
%%capture
!pip install sentence_transformers

In [12]:
from sentence_transformers import SentenceTransformer

In [13]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens') #한국어가 포함된 다국어 모델 로드

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
encoded = model.encode(data)

In [15]:
#임베딩 된 벡터 수 확인
len(encoded)

37648

In [21]:
%%capture
!pip install faiss-gpu

In [23]:
import faiss

In [25]:
#인덱스 정의
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
#인덱스에 데이터 추가
index.add_with_ids(encoded, np.array(range(0, len(data))))
faiss.write_index(index, 'classification')

In [26]:
def search(query):
   t = time.time()
   query_vector = model.encode([query])
   k = 5
   top_k = index.search(query_vector, k)
   print('total time: {}'.format(time.time() - t))
   return [data[_id] for _id in top_k[1].tolist()[0]]

In [28]:
query = str(input())
results = search(query)

print('results :')
for result in results:
   print('\t', result)

올해 수출 현황
total time: 0.06350111961364746
results :
	 우리나라는 수출의존도가 높은 편이지만 수입의존도도 만만치 않은 것으로 나타났다 이는 수출을 위한 원자재 수입 비중이 매우 큰 데 따른 것으로 대외경제에 크게 의존할 수밖에 없음을 보여주는 대목이다 특히 수입의존도 확대는 해외의 농산물 등 원자재 가격 등락에 우리나라 경제가 민감하게 반응할 수밖에 없음을 보여준다 수입의존도는 한 나라의 경제가 수입에 의존하는 정도로 평균 수입성향이라고도 한다 대외경제에 의존하고 있는지를 수입량으로 측정한 것이다 국민총소득GNI대비 백분율이다 IMG1 3일 한국은행에 따르면 우리나라 수입의존도는 지난해 554로 전년 513에 비해 41p 상승했다 2002년에 328였던 수입의존도는 2007년에 421로 뛰어올랐고 2008년에는 556로 무려 135p나 상승하면서 사상최고치를 기록했다 2009년에 473로 낮아진 후 2년연속 상승세를 이어가고 있다 7월 전체 수입품 중 원자재는 136억달러로 전체 수입액 423억달러의 321에 달했다 원유수입 도입단가가 전년 동월 배럴당 1117달러에서 1024달러로 떨어져 도입금액이 76 줄었다 국제유가와 도입단가가 곧바로 원자재 수입액을 크게 변화시킬 수밖에 없다 밀 옥수수 콩 등 세계 곡물가 급등이 47개월 후엔 수입을 통해 국내로 전염될 전망이다 높은 수출의존도는 외국의 경기침체나 경제위기가 국내로 전염되는 통로다 유럽의 재정위기 미국과 중국 등 선진국과 신흥국의 경기하강이 고스란히 세계 수요를 줄여 우리나라 상품이 수출하는 데 장애물로 작용하고 있다 수출의존도는 2002년 343에서 2004년에 421로 올라섰고 20052007년까지 부진하다 2008년에 551로 뛰어올랐다 지난해에는 577를 기록하며 위기 이전수준을 회복했다 수출과 수입의존도를 합한 무역의존도는 지난해 1132로 사상최고치를 기록했으며 2년연속 100를 넘어섰다
	 석유제품 철강판 호조  대일 만성적자 급감 4월 수출총액 사상 